In [27]:
from openai import OpenAI
client = OpenAI()
from tqdm import tqdm

In [13]:
def extract_markdown_blocks_with_code(file_path):
    """
    Extracts blocks from a Markdown file.
    A block starts with a line containing more than one '#' and ends at the next such line.
    Lines within code blocks are included in the running block.

    Returns a list of blocks, where each block is a single string.
    """
    blocks = []
    current_block = []
    inside_code_block = False
    block_started = False  # Flag to track if a block is in progress

    try:
        with open(file_path, 'r') as file:
            for line in file:
                stripped_line = line.strip()

                # Toggle code block detection
                if stripped_line.startswith('```'):
                    inside_code_block = not inside_code_block
                    # Include the ``` line in the current block
                    if block_started:
                        current_block.append(line.strip())
                    continue

                # Detect block start with more than one '#'
                if not inside_code_block and stripped_line.startswith('#') and len(stripped_line.split(' ')[0]) > 1:
                    if block_started:  # Save the current block if one is in progress
                        blocks.append('\n'.join(current_block))  # Convert the list to a string
                        current_block = []
                    block_started = True

                if block_started:
                    current_block.append(line.strip())

            # Append the last block if it exists
            if current_block:
                blocks.append('\n'.join(current_block))

        return blocks

    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []


In [28]:
def score_code_relevance(blocks):
    """
    Iterates through blocks, prompts GPT to score the relevance of code snippets to the text in the block.
    Skips blocks without code snippets.
    
    Args:
        blocks (list): A list of strings representing Markdown blocks.

    Returns:
        dict: A dictionary with block indices as keys and their relevance scores as values.
    """
    def call_gpt(prompt):
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            seed=42
        )
        try:
            score = int(response.choices[0].message.content)

        except:
            return None
        return score  # Assume the score for demonstration
    
    scores = {}

    # Iterate over blocks
    for i, block in tqdm(enumerate(blocks)):
        if '```' not in block:  # Skip blocks without code snippets
            continue
        
        # Construct the GPT prompt
        prompt = f"""
Below is a text and a code snippet. Rate the relevance of the code snippet to the text on a scale of 0 to 5, 
where 0 means no relevance and 5 means perfect relevance.

Text and Code Block:
{block}

Provide only the score as the response.
"""
        # Call GPT with the prompt
        score = call_gpt(prompt)
        if score is None:
            continue
        scores[i] = score  # Store the score with the block index as key

    return scores


In [40]:
blocks = extract_markdown_blocks_with_code('generated_blog.md')

In [41]:
scores = score_code_relevance(blocks)

94it [00:18,  5.00it/s]


In [47]:
average_score = sum([i for i in scores.values()]) / len(scores)
print(f"Average score on 5: {average_score}")
print(f"Average pecentage: {((average_score/5)*100)}%")

Average score on 5: 4.804347826086956
Average pecentage: 96.08695652173913%
